In [1]:
# Import SQLAlchemy `automap`, Flask, custom function and other dependencies. 
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func, desc
from flask import Flask, jsonify, render_template


# Database set up and connection
#import sqlalchemy
#from sqlalchemy.ext.automap import automap_base
#from sqlalchemy.orm import Session
#from sqlalchemy import create_engine, desc  #what else here???

#import custom functions
from bbd import return_sample_names

In [2]:
app = Flask(__name__)
      
engine = create_engine("sqlite:///DataSets/belly_button_biodiversity.sqlite")

Base = automap_base()
Base.prepare(engine, reflect=True)

In [3]:
Otu = Base.classes.otu
Samples = Base.classes.samples
Samples_metadata = Base.classes.samples_metadata

        
#create session 
session = Session(engine)

In [4]:
@app.route("/")
def home():
    #return render_template("index.html")
    return(
    f"Available Routes:<br/>"
    f"/names<br/>"
    f"/otu<br/>"
    f"/otu_descriptions<br/>"
    f"/metadata/<sample>"
    )


In [ ]:

@app.route("/names")
def names():
    sample_names = Samples.__table__.columns
    sample_names_ls = [name.key for name in sample_names]
    sample_names_ls.remove("otu_id")
    return jsonify(sample_names_ls)

@app.route("/otu")
def otu():
    otu_descriptions = session.query(Otu.lowest_taxonomic_unit_found).all()
    otu_descriptions_list = [x for (x), in otu_descriptions]
    return jsonify(otu_descriptions_list)

@app.route("/otu_descriptions")
def otu_disc():
    otu_descriptions = session.query(Otu.otu_id, Otu.lowest_taxonomic_unit_found).all()
    otu_dict = {}
    for row in otu_descriptions:
        otu_dict[row[0]] = row[1]
    return jsonify(otu_dict)

@app.route("/metadata/<sample>")
def sample_query(sample):
    sample_name = sample.replace("BB_", "")
    result = session.query(Samples_metadata.AGE, Samples_metadata.BBTYPE, Samples_metadata.ETHNICITY, Samples_metadata.GENDER, Samples_metadata.LOCATION, Samples_metadata.SAMPLEID).filter_by(SAMPLEID = sample_name).all()
    record = result[0]
    record_dict = {
        "AGE": record[0],
        "BBTYPE": record[1],
        "ETHNICITY": record[2],
        "GENDER": record[3],
        "LOCATION": record[4],
        "SAMPLEID": record[5]
    }
    return jsonify(record_dict)

@app.route('/wfreq/<sample>')
def wash_freq(sample):
    sample_name = sample.replace("BB_", "")
    result = session.query(Samples_metadata.WFREQ).filter_by(SAMPLEID = sample_name).all()
    wash_freq = result[0][0]
    return jsonify(wash_freq)

@app.route('/samples/<sample>')
def otu_data(sample):
    sample_query = "Samples." + sample
    result = session.query(Samples.otu_id, sample_query).order_by(desc(sample_query)).all()
    otu_ids = [result[x][0] for x in range(len(result))]   
    sample_values = [result[x][1] for x in range(len(result))]
    dict_list = [{"otu_ids": otu_ids}, {"sample_values": sample_values}]
    return jsonify(dict_list)

if __name__ == '__main__':
    app.run(debug=False)
    #changed from True to False
# if __name__ == '__main__':
#   climate_analysis_app.run(debug=False)

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [20/May/2018 13:18:10] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/May/2018 13:18:10] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [20/May/2018 13:18:10] "GET /robots.txt HTTP/1.1" 404 -
127.0.0.1 - - [20/May/2018 13:18:15] "GET /names HTTP/1.1" 200 -
127.0.0.1 - - [20/May/2018 13:19:07] "GET /otu HTTP/1.1" 200 -
127.0.0.1 - - [20/May/2018 13:19:57] "GET /otu_descriptions HTTP/1.1" 200 -
127.0.0.1 - - [20/May/2018 13:20:06] "GET /metadata/ HTTP/1.1" 404 -
127.0.0.1 - - [20/May/2018 13:20:31] "GET /names HTTP/1.1" 200 -
127.0.0.1 - - [20/May/2018 13:20:51] "GET /otu HTTP/1.1" 200 -
127.0.0.1 - - [20/May/2018 13:20:51] "GET /robots.txt HTTP/1.1" 404 -
127.0.0.1 - - [20/May/2018 13:22:54] "GET /metadata/940 HTTP/1.1" 200 -
127.0.0.1 - - [20/May/2018 13:24:02] "GET /wfreq/940 HTTP/1.1" 200 -
127.0.0.1 - - [20/May/2018 13:25:31] "GET /sample/940 HTTP/1.1" 404 -
[2018-05-20 13:25:40,644] ERROR in app: Exception on /sam

127.0.0.1 - - [20/May/2018 13:26:34] "GET /metadata/940 HTTP/1.1" 200 -
127.0.0.1 - - [20/May/2018 13:27:18] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/May/2018 13:27:39] "GET /names HTTP/1.1" 200 -
127.0.0.1 - - [20/May/2018 13:28:28] "GET /otu HTTP/1.1" 200 -
[2018-05-20 13:29:13,690] ERROR in app: Exception on /samples/940 [GET]
Traceback (most recent call last):
  File "C:\Users\nisha\Anaconda\lib\site-packages\sqlalchemy\engine\base.py", line 1193, in _execute_context
    context)
  File "C:\Users\nisha\Anaconda\lib\site-packages\sqlalchemy\engine\default.py", line 507, in do_execute
    cursor.execute(statement, parameters)
sqlite3.OperationalError: near ".940": syntax error

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\nisha\Anaconda\lib\site-packages\flask\app.py", line 1982, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\nisha\Anaconda\lib\site-packages\flask\app.py", line 1614

In [ ]:
#%tb

In [ ]:
# app = QApplication(sys.argv)
# app.aboutToQuit.connect(app.deleteLater)